In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [2]:
test_data = pd.read_csv('test.csv')
train_data = pd.read_csv('train.csv')

In [3]:
# plt.scatter(test_data.Rooms, test_data.Square)

In [4]:
# Функции для очистки и подготовки данных
mean_year = np.round(train_data.loc[train_data['HouseYear'] <= 2020, 'HouseYear'].mean())
mean_healthcare = np.round(train_data["Healthcare_1"].mean())
mean_square_for_max = train_data.loc[(train_data['Rooms'] <= train_data.loc[(train_data['Square'] > 300), 'Rooms'].mean()), 'Square'].mean()
mean_square_for_big_ls = train_data.loc[train_data['LifeSquare'] > 250, 'Square'].mean()
mean_life_squae_for_max = train_data.loc[train_data['Square'] >= mean_square_for_big_ls, 'LifeSquare'].mean()

In [5]:
def clean_year(df, mean_year):
    df.loc[df['HouseYear'] > 2020, 'HouseYear'] = mean_year

In [6]:
def clean_life_square(df, koef_S_LS):
    df.loc[(df['LifeSquare'] < 21) | (df['LifeSquare'].isnull()), 'LifeSquare'] = df['Square']*0.85
    df.loc[df['LifeSquare'] > 250, 'LifeSquare'] = mean_life_squae_for_max

In [7]:
def clean_square(df, mean_square_for_max):
    df.loc[(df['Square'] > 300), 'Square'] = mean_square_for_max

In [8]:
def clean_healthcare_1(df, mean_healthcare):
    df.loc[df['Healthcare_1'].isnull(), 'Healthcare_1'] = mean_healthcare

In [9]:
def clean_rooms(df):
    df.loc[(df['Rooms'] < 1) & (df['LifeSquare'] < 30), 'Rooms'] = 1
    df.loc[(df['Rooms'] < 1) & (df['LifeSquare'] > 30) & (df['LifeSquare'] < 45), 'Rooms'] = 2
    df.loc[(df['Rooms'] < 1) & (df['LifeSquare'] > 45) & (df['LifeSquare'] < 60), 'Rooms'] = 3
    df.loc[(df['Rooms'] < 1) & (df['LifeSquare'] > 60) & (df['LifeSquare'] < 75), 'Rooms'] = 4
    df.loc[(df['Rooms'] < 1) & (df['LifeSquare'] > 70), 'Rooms'] = 6
    df.loc[(df['Rooms'] > 10), 'Rooms'] = 2

In [10]:
def prepare_data(df, mean_year=mean_year, mean_healthcare=mean_healthcare, mean_square_for_max=mean_square_for_max, mean_life_squae_for_max=mean_life_squae_for_max):
    clean_year(df, mean_year)
    clean_life_square(df, mean_life_squae_for_max)
    clean_healthcare_1(df, mean_healthcare)
    clean_rooms(df)
    clean_square(df, mean_square_for_max)

In [11]:
prepare_data(train_data)
prepare_data(test_data)

In [12]:
X = pd.get_dummies(train_data)
X.drop("Price", axis=1, inplace=True)
X.drop("Id", axis=1, inplace=True)
y = train_data.Price

In [13]:
# df_num_features = train_data.select_dtypes(include=['float64','uint8'])  # отбираем количественные признаки
# import seaborn as sns
# sns.pairplot(df_num_features);

# GradientBoostingRegressor

In [14]:
test_data = pd.read_csv('test.csv')
train_data = pd.read_csv('train.csv')
prepare_data(train_data)
prepare_data(test_data)
X = pd.get_dummies(train_data)
X.drop("Price", axis=1, inplace=True)
X.drop("Id", axis=1, inplace=True)
y = train_data.Price

In [35]:
k=0
from sklearn.ensemble import GradientBoostingRegressor
for number in range(25):
    k=round(k+1,2)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.16, random_state=42)
    
    # переобучение и оценка модели
    def clean_life_square(df, koef_S_LS):
        df.loc[(df['LifeSquare'] < k) | (df['LifeSquare'].isnull()), 'LifeSquare'] = df['Square']*0.5
        df.loc[df['LifeSquare'] > 250, 'LifeSquare'] = mean_life_squae_for_max
    
#     def KitchenSquare(df, mean_square_for_max):
#         df.loc[(df['KitchenSquare'] < 3) | (df['KitchenSquare'].isnull()), 'KitchenSquare'] = mean_square_Kitchen
#         df.loc[(df['KitchenSquare'] > k)] = mean_square_Kitchen
    
#     def clean_square(df, mean_square_for_max):
#         df.loc[(df['Square'] > 300), 'Square'] = mean_square_for_max


    
    test_data = pd.read_csv('test.csv')
    train_data = pd.read_csv('train.csv')
    prepare_data(train_data)
    prepare_data(test_data)
    X = pd.get_dummies(train_data)
    X.drop("Price", axis=1, inplace=True)
    X.drop("Id", axis=1, inplace=True)
    y = train_data.Price
#     print(train_data.LifeSquare.sum())
    
    final_model = GradientBoostingRegressor(n_estimators=200, max_depth=5, random_state=42
                                           )
    # min_samples_split=5, subsample=0.5 , min_samples_leaf=4 

    final_model.fit(X_train, y_train)

    y_pred_gbr = final_model.predict(X_valid)
    y_pred_train_gbr = final_model.predict(X_train)

    print('r2: ', r2_score(y_valid, y_pred_gbr),', KitchenSquare: ',k)

r2:  0.754177566833658 , KitchenSquare:  1
r2:  0.7554334494911744 , KitchenSquare:  2
r2:  0.7470031316294251 , KitchenSquare:  3
r2:  0.7611377586076447 , KitchenSquare:  4
r2:  0.7566267936577936 , KitchenSquare:  5
r2:  0.7537185452061099 , KitchenSquare:  6
r2:  0.7572424880521106 , KitchenSquare:  7
r2:  0.7572424880521106 , KitchenSquare:  8
r2:  0.7572424880521106 , KitchenSquare:  9
r2:  0.7595701141229902 , KitchenSquare:  10
r2:  0.7595701141229902 , KitchenSquare:  11
r2:  0.7637329092455465 , KitchenSquare:  12
r2:  0.7562348218489922 , KitchenSquare:  13
r2:  0.7608988773408569 , KitchenSquare:  14
r2:  0.7596691106640145 , KitchenSquare:  15
r2:  0.7595734040664491 , KitchenSquare:  16
r2:  0.7579921575364381 , KitchenSquare:  17
r2:  0.7532676414870421 , KitchenSquare:  18
r2:  0.7548560796297121 , KitchenSquare:  19
r2:  0.7545349353927457 , KitchenSquare:  20
r2:  0.7537968769269786 , KitchenSquare:  21
r2:  0.754177566833658 , KitchenSquare:  22
r2:  0.75428734838338

In [33]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.16, random_state=42)
# переобучение и оценка модели
from sklearn.ensemble import GradientBoostingRegressor
final_model = GradientBoostingRegressor(n_estimators=200, max_depth=5, random_state=42
                                       )
# min_samples_split=5, subsample=0.5 , min_samples_leaf=4 

final_model.fit(X_train, y_train)

y_pred_gbr = final_model.predict(X_valid)
y_pred_train_gbr = final_model.predict(X_train)

r2_score(y_valid, y_pred_gbr)

0.7577861874526557

In [17]:
# Предсказываем цены для тестовых данных и выгружаем в файл
X_test = pd.get_dummies(test_data)
X_test.drop("Id", axis=1, inplace=True)
test_data["Price"] = final_model.predict(X_test)

In [18]:
# экспорт в файл
test_data.loc[:, ['Id', 'Price']].to_csv('best_gbr_02.csv', index=False)

0.7621056876187297 - test_size=0.16 - n_estimators=200, max_depth=5, random_state=42 (0.75339)
0.75142


In [16]:
# k=1000
# for number in range(200):
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.16, random_state=42)
#     # переобучение и оценка модели
#     from sklearn.ensemble import GradientBoostingRegressor
#     final_model = GradientBoostingRegressor(n_estimators=200, max_depth=5, random_state=42
#                                            )
#     # min_samples_split=5, subsample=0.5 , min_samples_leaf=4 

#     final_model.fit(X_train, y_train)

#     y_pred_gbr = final_model.predict(X_valid)
#     y_pred_train_gbr = final_model.predict(X_train)

#     print('r2: ', r2_score(y_valid, y_pred),', n_estimators: ',k)

In [71]:
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse
import seaborn as sns
def evaluate_preds(true_values, pred_values):
    print("R2:\t" + str(round(r2(true_values, pred_values), 9)) + "\n" +
          "MAE:\t" + str(round(mae(true_values, pred_values), 9)) + "\n" +
          "MSE:\t" + str(round(mse(true_values, pred_values), 9)))
    
    plt.figure(figsize=(10,10)) 
    
    sns.scatterplot(x=pred_values, y=true_values)
    
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('True vs Predicted values')
    plt.show()

In [76]:
# y_train_preds = final_model.predict(X_train)
# evaluate_preds(y_train, y_train_preds)